In [4]:
a = fread("human_gene_name.tsv")
a$n <- 1
fwrite(a, "human_gene_name.tsv", sep = "\t")

In [ ]:
sos run xqtl-protocol/code/data_preprocessing/phenotype/gene_annotation.ipynb annotate_coord \
    --cwd ./ \
    --phenoFile human_gene_name.tsv \
    --phenotype-id-column gene_name \
    --molecular-trait-type gene \
    --coordinate-annotation data/resource/references/Homo_sapiens.GRCh38.103.chr.reformatted.collapse_only.gene.ERCC.gtf

In [12]:
library(tidyverse)
library(data.table)

In [13]:
setwd("~/")
getwd()

[1] "/home/ubuntu"

## sQTL

In [34]:
meta_file = fread("project/image_QTL/DNE_cb_explained_by_xQTL/xQTL_top_loci.tsv")
meta_file = meta_file |> filter(Data_Type == "sQTL")
dim(meta_file)

[1] 3 5

In [35]:
# Assuming your meta_file dataframe is already loaded
# If not, you'd need to recreate it from your data

# Create a function to read a single file and add identifiers
read_topolci_file <- function(path, cohort, data_type, modality) {
  # Read the file - assuming tab-delimited with gzip compression
  data <- read_tsv(path, show_col_types = FALSE)
  
  # Add identifier columns
  data$Cohort <- cohort
  data$Data_Type <- data_type
  data$Modality <- modality
  
  # Create combined identifier
  data$Cohort_Data_Type_Modality <- paste(cohort, data_type, modality, sep = "_")
  
  return(data)
}

# Apply the function to each row in the meta_file
combined_data <- pmap_dfr(
  list(
    path = meta_file$Path,
    cohort = meta_file$Cohort,
    data_type = meta_file$Data_Type,
    modality = meta_file$Modality
  ),
  read_topolci_file
)

# Save the combined dataset if needed
#fwrite(combined_data, "combined_top_loci_data.tsv")

# Preview the result
head(combined_data)

#chr,start,end,a1,a2,variant_ID,gene_ID,event_ID,cs_coverage_0.95,cs_coverage_0.7,cs_coverage_0.5,PIP,conditional_effect,cluster,cat,ID,Cohort,Data_Type,Modality,Cohort_Data_Type_Modality
<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,10396,10397,*,CCCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCCTAACCCTAACCCTAACCCTAACCCTCGCGGTACCCTCAG,chr1:10397:CCCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCCTAACCCTAACCCTAACCCTAACCCTCGCGGTACCCTCAG:*,ENSG00000158286,ROSMAP_AC_sQTL_chr1:6209539:6209928:clu_403_+:PR:ENSG00000158286,0,0,0,0.02901275,0.4162247,clu_403_,PR,chr1:6209539:6209928:clu_403_+:PR,ROSMAP,sQTL,AC,ROSMAP_sQTL_AC
1,10396,10397,*,CCCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCCTAACCCTAACCCTAACCCTAACCCTCGCGGTACCCTCAG,chr1:10397:CCCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCCTAACCCTAACCCTAACCCTAACCCTCGCGGTACCCTCAG:*,ENSG00000248333,ROSMAP_AC_sQTL_chr1:1637202:1637407:clu_6616_-:PR:ENSG00000248333,0,0,0,0.04186011,-0.6134922,clu_6616_,PR,chr1:1637202:1637407:clu_6616_-:PR,ROSMAP,sQTL,AC,ROSMAP_sQTL_AC
1,10432,10433,ACCCTTAACCCCTAACCCCTAACC,ACCCTAACC,chr1:10433:ACCCTAACC:ACCCTTAACCCCTAACCCCTAACC,ENSG00000116213,ROSMAP_AC_sQTL_chr1:3647560:3649930:clu_6810_-:PR:ENSG00000116213,0,0,0,0.08295409,-1.2588737,clu_6810_,PR,chr1:3647560:3649930:clu_6810_-:PR,ROSMAP,sQTL,AC,ROSMAP_sQTL_AC
1,10437,10438,*,AAC,chr1:10438:AAC:*,ENSG00000160087,ROSMAP_AC_sQTL_chr1:1256181:1256991:clu_6523_-:UP:ENSG00000160087,0,0,0,0.02530852,0.7696662,clu_6523_,UP,chr1:1256181:1256991:clu_6523_-:UP,ROSMAP,sQTL,AC,ROSMAP_sQTL_AC
1,10439,10440,*,CCCCTAACCCTAACCCTAACCCTAACCCTCGCGGTACCCTCAG,chr1:10440:CCCCTAACCCTAACCCTAACCCTAACCCTCGCGGTACCCTCAG:*,ENSG00000078369,ROSMAP_AC_sQTL_chr1:1839238:1839628:clu_6656_-:NE:ENSG00000078369,0,0,0,0.02845004,-0.2279109,clu_6656_,NE,chr1:1839238:1839628:clu_6656_-:NE,ROSMAP,sQTL,AC,ROSMAP_sQTL_AC
1,10442,10443,TTAACCCTAACCCTAACCCT,CTAACCCTAACCCTAACCCT,chr1:10443:CTAACCCTAACCCTAACCCT:TTAACCCTAACCCTAACCCT,ENSG00000169972,ROSMAP_AC_sQTL_chr1:1309851:1310633:clu_127_+:PR:ENSG00000169972,0,0,0,0.02746254,-1.3924732,clu_127_,PR,chr1:1309851:1310633:clu_127_+:PR,ROSMAP,sQTL,AC,ROSMAP_sQTL_AC


In [52]:
three_overlap = readLines("project/image_QTL/interested_genes/CB_M1/DDX17_CBY1_var.txt")
three_overlap <- gsub('"', '', three_overlap)

In [53]:
three_overlap

[1] "chr22:38578149:C:A"     "chr22:38590243:G:A"     "chr22:38592110:A:G"    
 [4] "chr22:38593808:A:G"     "chr22:38597462:A:G"     "chr22:38599380:G:T"    
 [7] "chr22:38602514:C:A"     "chr22:38609057:A:T"     "chr22:38617825:T:C"    
[10] "chr22:38625830:G:GC"    "chr22:38640275:T:C"     "chr22:38640764:T:C"    
[13] "chr22:38641699:T:TGTTG" "chr22:38649094:T:C"     "chr22:38652311:A:G"    
[16] "chr22:38657752:G:C"     "chr22:38676093:C:G"     "chr22:38677891:A:G"    
[19] "chr22:38684073:T:C"     "chr22:38686169:A:G"     "chr22:38703597:T:C"    
[22] "chr22:38705614:G:C"     "chr22:38705628:T:C"     "chr22:38708315:C:T"    
[25] "chr22:38709943:T:C"     "chr22:38717129:G:A"     "chr22:38717898:C:T"    
[28] "chr22:38720790:A:C"     "chr22:38733994:A:G"     "chr22:38734959:C:T"    
[31] "chr22:38737798:C:T"     "chr22:38738710:T:C"     "chr22:38741829:C:G"    
[34] "chr22:38743338:A:G"     "chr22:38745595:G:A"     "chr22:38750522:C:T"    
[37] "chr22:38751710:C:T"     "chr22:38752478:G:A"     "chr22:38754445:G:A"    
[40] "chr22:38754538:G:T"

In [59]:
SUN2_var = combined_data |> filter(variant_ID %in% three_overlap)
SUN2_var |> count(gene_ID)

gene_ID,n
<chr>,<int>
ENSG00000100211,5
ENSG00000100221,4
ENSG00000184949,109
ENSG00000198792,23


In [55]:
SUN2_var

#chr,start,end,a1,a2,variant_ID,gene_ID,event_ID,cs_coverage_0.95,cs_coverage_0.7,cs_coverage_0.5,PIP,conditional_effect,cluster,cat,ID,Cohort,Data_Type,Modality,Cohort_Data_Type_Modality
<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
22,38592109,38592110,G,A,chr22:38592110:A:G,ENSG00000100211,ROSMAP_AC_sQTL_chr22:38668132:38670766:clu_143018_+:UP:ENSG00000100211,0,0,0,0.021934321,-0.40321481,clu_143018_,UP,chr22:38668132:38670766:clu_143018_+:UP,ROSMAP,sQTL,AC,ROSMAP_sQTL_AC
22,38592109,38592110,G,A,chr22:38592110:A:G,ENSG00000100211,ROSMAP_AC_sQTL_chr22:38670989:38673158:clu_143019_+:UP:ENSG00000100211,0,0,0,0.001049308,-0.25552230,clu_143019_,UP,chr22:38670989:38673158:clu_143019_+:UP,ROSMAP,sQTL,AC,ROSMAP_sQTL_AC
22,38592109,38592110,G,A,chr22:38592110:A:G,ENSG00000100221,ROSMAP_AC_sQTL_chr22:38700123:38700965:clu_144722_-:NE:ENSG00000100221,0,0,0,0.002629362,0.04866929,clu_144722_,NE,chr22:38700123:38700965:clu_144722_-:NE,ROSMAP,sQTL,AC,ROSMAP_sQTL_AC
22,38597461,38597462,G,A,chr22:38597462:A:G,ENSG00000100211,ROSMAP_AC_sQTL_chr22:38668132:38670766:clu_143018_+:UP:ENSG00000100211,0,0,0,0.001920936,-0.35379653,clu_143018_,UP,chr22:38668132:38670766:clu_143018_+:UP,ROSMAP,sQTL,AC,ROSMAP_sQTL_AC
22,38597461,38597462,G,A,chr22:38597462:A:G,ENSG00000100211,ROSMAP_AC_sQTL_chr22:38670989:38673158:clu_143019_+:UP:ENSG00000100211,0,0,0,0.001279521,-0.27017207,clu_143019_,UP,chr22:38670989:38673158:clu_143019_+:UP,ROSMAP,sQTL,AC,ROSMAP_sQTL_AC
22,38617824,38617825,C,T,chr22:38617825:T:C,ENSG00000100211,ROSMAP_AC_sQTL_chr22:38668132:38670766:clu_143018_+:UP:ENSG00000100211,0,0,0,0.001075891,-0.31508327,clu_143018_,UP,chr22:38668132:38670766:clu_143018_+:UP,ROSMAP,sQTL,AC,ROSMAP_sQTL_AC
22,38717128,38717129,A,G,chr22:38717129:G:A,ENSG00000100221,ROSMAP_AC_sQTL_chr22:38700123:38700965:clu_144722_-:NE:ENSG00000100221,0,0,0,0.002035720,0.04965771,clu_144722_,NE,chr22:38700123:38700965:clu_144722_-:NE,ROSMAP,sQTL,AC,ROSMAP_sQTL_AC
22,38717897,38717898,T,C,chr22:38717898:C:T,ENSG00000100221,ROSMAP_AC_sQTL_chr22:38700123:38700965:clu_144722_-:NE:ENSG00000100221,0,0,0,0.002035720,0.04965771,clu_144722_,NE,chr22:38700123:38700965:clu_144722_-:NE,ROSMAP,sQTL,AC,ROSMAP_sQTL_AC
22,38754444,38754445,A,G,chr22:38754445:G:A,ENSG00000100221,ROSMAP_AC_sQTL_chr22:38700123:38700965:clu_144722_-:NE:ENSG00000100221,0,0,0,0.002122077,0.04997759,clu_144722_,NE,chr22:38700123:38700965:clu_144722_-:NE,ROSMAP,sQTL,AC,ROSMAP_sQTL_AC


In [56]:
CBY1_var = combined_data |> filter(variant_ID %in% three_overlap)

In [58]:
CBY1_var |> filter(gene_ID == "ENSG00000100211")

#chr,start,end,a1,a2,variant_ID,gene_ID,event_ID,cs_coverage_0.95,cs_coverage_0.7,cs_coverage_0.5,PIP,conditional_effect,cluster,cat,ID,Cohort,Data_Type,Modality,Cohort_Data_Type_Modality
<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
22,38592109,38592110,G,A,chr22:38592110:A:G,ENSG00000100211,ROSMAP_AC_sQTL_chr22:38668132:38670766:clu_143018_+:UP:ENSG00000100211,0,0,0,0.021934321,-0.4032148,clu_143018_,UP,chr22:38668132:38670766:clu_143018_+:UP,ROSMAP,sQTL,AC,ROSMAP_sQTL_AC
22,38592109,38592110,G,A,chr22:38592110:A:G,ENSG00000100211,ROSMAP_AC_sQTL_chr22:38670989:38673158:clu_143019_+:UP:ENSG00000100211,0,0,0,0.001049308,-0.2555223,clu_143019_,UP,chr22:38670989:38673158:clu_143019_+:UP,ROSMAP,sQTL,AC,ROSMAP_sQTL_AC
22,38597461,38597462,G,A,chr22:38597462:A:G,ENSG00000100211,ROSMAP_AC_sQTL_chr22:38668132:38670766:clu_143018_+:UP:ENSG00000100211,0,0,0,0.001920936,-0.3537965,clu_143018_,UP,chr22:38668132:38670766:clu_143018_+:UP,ROSMAP,sQTL,AC,ROSMAP_sQTL_AC
22,38597461,38597462,G,A,chr22:38597462:A:G,ENSG00000100211,ROSMAP_AC_sQTL_chr22:38670989:38673158:clu_143019_+:UP:ENSG00000100211,0,0,0,0.001279521,-0.2701721,clu_143019_,UP,chr22:38670989:38673158:clu_143019_+:UP,ROSMAP,sQTL,AC,ROSMAP_sQTL_AC
22,38617824,38617825,C,T,chr22:38617825:T:C,ENSG00000100211,ROSMAP_AC_sQTL_chr22:38668132:38670766:clu_143018_+:UP:ENSG00000100211,0,0,0,0.001075891,-0.3150833,clu_143018_,UP,chr22:38668132:38670766:clu_143018_+:UP,ROSMAP,sQTL,AC,ROSMAP_sQTL_AC


In [67]:
DDX17_var = combined_data |> filter(variant_ID %in% three_overlap)

In [71]:
DDX17_var |> filter(gene_ID == "ENSG00000100201")

#chr,start,end,a1,a2,variant_ID,gene_ID,event_ID,cs_coverage_0.95,cs_coverage_0.7,cs_coverage_0.5,PIP,conditional_effect,cluster,cat,ID,Cohort,Data_Type,Modality,Cohort_Data_Type_Modality
<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>


## eQTL

In [28]:
meta_file = fread("project/image_QTL/DNE_cb_explained_by_xQTL/xQTL_top_loci.tsv")
meta_file = meta_file |> filter(Data_Type == "eQTL" & Cohort == "ROSMAP" & Method == "single_context_finemapping")
meta_file

Data_Type,Cohort,Modality,Method,Path
<chr>,<chr>,<chr>,<chr>,<chr>
eQTL,ROSMAP,AC,single_context_finemapping,data/analysis_result/single_context/ROSMAP_eQTL/export/summary/context_specific/AC_DeJager_eQTL.exported.toploci.bed.gz
eQTL,ROSMAP,Ast_10_Kellis,single_context_finemapping,data/analysis_result/single_context/ROSMAP_eQTL/export/summary/context_specific/Ast_10_Kellis_eQTL.exported.toploci.bed.gz
eQTL,ROSMAP,Ast_DeJager,single_context_finemapping,data/analysis_result/single_context/ROSMAP_eQTL/export/summary/context_specific/Ast_DeJager_eQTL.exported.toploci.bed.gz
eQTL,ROSMAP,Ast_Kellis,single_context_finemapping,data/analysis_result/single_context/ROSMAP_eQTL/export/summary/context_specific/Ast_Kellis_eQTL.exported.toploci.bed.gz
eQTL,ROSMAP,Ast_mega,single_context_finemapping,data/analysis_result/single_context/ROSMAP_eQTL/export/summary/context_specific/Ast_mega_eQTL.exported.toploci.bed.gz
eQTL,ROSMAP,DLPFC,single_context_finemapping,data/analysis_result/single_context/ROSMAP_eQTL/export/summary/context_specific/DLPFC_DeJager_eQTL.exported.toploci.bed.gz
eQTL,ROSMAP,Exc_DeJager,single_context_finemapping,data/analysis_result/single_context/ROSMAP_eQTL/export/summary/context_specific/Exc_DeJager_eQTL.exported.toploci.bed.gz
eQTL,ROSMAP,Exc_Kellis,single_context_finemapping,data/analysis_result/single_context/ROSMAP_eQTL/export/summary/context_specific/Exc_Kellis_eQTL.exported.toploci.bed.gz
eQTL,ROSMAP,Exc_mega,single_context_finemapping,data/analysis_result/single_context/ROSMAP_eQTL/export/summary/context_specific/Exc_mega_eQTL.exported.toploci.bed.gz


In [29]:
# Assuming your meta_file dataframe is already loaded
# If not, you'd need to recreate it from your data

# Create a function to read a single file and add identifiers
read_topolci_file <- function(path, cohort, data_type, modality) {
  # Read the file - assuming tab-delimited with gzip compression
  data <- read_tsv(path, show_col_types = FALSE)
  
  # Add identifier columns
  data$Cohort <- cohort
  data$Data_Type <- data_type
  data$Modality <- modality
  
  # Create combined identifier
  data$Cohort_Data_Type_Modality <- paste(cohort, data_type, modality, sep = "_")
  
  return(data)
}

# Apply the function to each row in the meta_file
combined_data <- pmap_dfr(
  list(
    path = meta_file$Path,
    cohort = meta_file$Cohort,
    data_type = meta_file$Data_Type,
    modality = meta_file$Modality
  ),
  read_topolci_file
)

# Save the combined dataset if needed
#fwrite(combined_data, "combined_top_loci_data.tsv")

# Preview the result
head(combined_data)

#chr,start,end,a1,a2,variant_ID,gene_ID,event_ID,cs_coverage_0.95,cs_coverage_0.7,cs_coverage_0.5,PIP,conditional_effect,Cohort,Data_Type,Modality,Cohort_Data_Type_Modality
<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>
1,99088,99089,*,TTTC,chr1:99089:TTTC:*,ENSG00000196581,AC_DeJager_eQTL_ENSG00000196581,0,0,0,0.14741504,0.3104743,ROSMAP,eQTL,AC,ROSMAP_eQTL_AC
1,600805,600806,C,G,chr1:600806:G:C,ENSG00000198912,AC_DeJager_eQTL_ENSG00000198912,0,0,0,0.23786542,-1.0948060,ROSMAP,eQTL,AC,ROSMAP_eQTL_AC
1,602262,602263,*,ATCC,chr1:602263:ATCC:*,ENSG00000187730,AC_DeJager_eQTL_ENSG00000187730,0,0,0,0.03324526,0.3465442,ROSMAP,eQTL,AC,ROSMAP_eQTL_AC
1,602277,602278,*,TCCCTCTCCTTTCTCCTCTCTAGCC,chr1:602278:TCCCTCTCCTTTCTCCTCTCTAGCC:*,ENSG00000187730,AC_DeJager_eQTL_ENSG00000187730,0,0,0,0.07769440,0.3721831,ROSMAP,eQTL,AC,ROSMAP_eQTL_AC
1,610727,610728,T,C,chr1:610728:C:T,ENSG00000127054,AC_DeJager_eQTL_ENSG00000127054,0,0,0,0.04314056,-0.4321317,ROSMAP,eQTL,AC,ROSMAP_eQTL_AC
1,618361,618362,A,G,chr1:618362:G:A,ENSG00000196581,AC_DeJager_eQTL_ENSG00000196581,0,0,0,0.02578836,-0.0710322,ROSMAP,eQTL,AC,ROSMAP_eQTL_AC


In [31]:
a = combined_data |> filter(str_detect(event_ID, "ENSG00000100211"))
a |> filter(variant_ID %in% var_PR_id)

#chr,start,end,a1,a2,variant_ID,gene_ID,event_ID,cs_coverage_0.95,cs_coverage_0.7,cs_coverage_0.5,PIP,conditional_effect,Cohort,Data_Type,Modality,Cohort_Data_Type_Modality
<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>


## TWAS

In [4]:
image_Aging3_mega = fread("project/image_QTL/twas/output/image_Aging3_ROSMAP_mega_eQTL/twas/image_Aging3_ROSMAP_mega.chr22_37320379_39179317.twas.tsv.gz")
image_Aging3_sQTL = fread("project/image_QTL/twas/output/image_Aging3_ROSMAP_sQTL/twas/image_Aging3_ROSMAP_sQTL.chr22_37320379_39179317.twas.tsv.gz")

In [5]:
image_Aging3_twas = rbind(image_Aging3_mega, image_Aging3_sQTL)

In [6]:
head(image_Aging3_twas)
dim(image_Aging3_twas)

chr,molecular_id,TSS,start,end,context,gwas_study,method,is_imputable,is_selected_method,rsq_cv,pval_cv,twas_z,twas_pval,type,block
<int>,<chr>,<int>,<int>,<int>,<chr>,<chr>,<chr>,<lgl>,<lgl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
22,ENSG00000100083,37608724,35240000,41680000,Ast_mega_eQTL,image_Aging3,enet,TRUE,FALSE,0.0008218826,0.2055405066,-0.4376689,0.6616263,eQTL,chr22_37320379_39179317
22,ENSG00000100083,37608724,35240000,41680000,Ast_mega_eQTL,image_Aging3,lasso,TRUE,FALSE,-0.0004507627,0.4138790725,-0.4687959,0.6392155,eQTL,chr22_37320379_39179317
22,ENSG00000100083,37608724,35240000,41680000,Ast_mega_eQTL,image_Aging3,mrash,TRUE,TRUE,0.0182399059,0.0001387118,-0.1555787,0.8763651,eQTL,chr22_37320379_39179317
22,ENSG00000100083,37608724,35240000,41680000,Ast_mega_eQTL,image_Aging3,susie,TRUE,FALSE,0.0083388917,0.0074995657,-0.1157768,0.9078294,eQTL,chr22_37320379_39179317
22,ENSG00000100083,37608724,35240000,41680000,Exc_mega_eQTL,image_Aging3,enet,TRUE,FALSE,0.0022901271,0.1014455687,-0.4423217,0.6582564,eQTL,chr22_37320379_39179317
22,ENSG00000100083,37608724,35240000,41680000,Exc_mega_eQTL,image_Aging3,lasso,TRUE,FALSE,-0.0002490971,0.3664398917,-1.0288466,0.3035518,eQTL,chr22_37320379_39179317


[1] 1144   16

In [7]:
# DDX17
image_Aging3_twas |> filter(molecular_id == "ENSG00000100201"& twas_pval < 0.025)


chr,molecular_id,TSS,start,end,context,gwas_study,method,is_imputable,is_selected_method,rsq_cv,pval_cv,twas_z,twas_pval,type,block
<int>,<chr>,<int>,<int>,<int>,<chr>,<chr>,<chr>,<lgl>,<lgl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
22,ENSG00000100201,38507659,37483438,41680000,Ast_mega_eQTL,image_Aging3,enet,TRUE,FALSE,0.0190624461,1.000798e-04,-3.275110,0.0010562085,eQTL,chr22_37320379_39179317
22,ENSG00000100201,38507659,37483438,41680000,Ast_mega_eQTL,image_Aging3,lasso,TRUE,TRUE,0.0210474022,4.559495e-05,-2.952844,0.0031486069,eQTL,chr22_37320379_39179317
22,ENSG00000100201,38507659,37483438,41680000,Ast_mega_eQTL,image_Aging3,mrash,TRUE,FALSE,0.0086394776,6.625041e-03,-3.531982,0.0004124576,eQTL,chr22_37320379_39179317
22,ENSG00000100201,38507659,37483438,41680000,Ast_mega_eQTL,image_Aging3,susie,TRUE,FALSE,0.0121631394,1.576276e-03,-3.106612,0.0018924487,eQTL,chr22_37320379_39179317
22,ENSG00000100201,38507659,37483438,41680000,Inh_mega_eQTL,image_Aging3,enet,TRUE,FALSE,0.0009835133,1.896414e-01,-2.499876,0.0124236829,eQTL,chr22_37320379_39179317
22,ENSG00000100201,38507659,37483438,41680000,Inh_mega_eQTL,image_Aging3,lasso,TRUE,FALSE,0.0009841834,1.895781e-01,-2.456518,0.0140290945,eQTL,chr22_37320379_39179317
22,ENSG00000100201,38507659,37483438,41680000,OPC_mega_eQTL,image_Aging3,enet,TRUE,FALSE,0.0092237281,5.264166e-03,-2.999911,0.0027005858,eQTL,chr22_37320379_39179317
22,ENSG00000100201,38507659,37483438,41680000,OPC_mega_eQTL,image_Aging3,lasso,TRUE,FALSE,0.0064412913,1.665710e-02,-2.926911,0.0034234681,eQTL,chr22_37320379_39179317
22,ENSG00000100201,38507659,37483438,41680000,OPC_mega_eQTL,image_Aging3,mrash,TRUE,FALSE,0.0099631871,3.892536e-03,-2.507823,0.0121477644,eQTL,chr22_37320379_39179317


In [8]:
# CBY1
image_Aging3_twas |> filter(molecular_id == "ENSG00000100211" & str_detect(type, "eQTL") & twas_pval < 4*1e-3) 

chr,molecular_id,TSS,start,end,context,gwas_study,method,is_imputable,is_selected_method,rsq_cv,pval_cv,twas_z,twas_pval,type,block
<int>,<chr>,<int>,<int>,<int>,<chr>,<chr>,<chr>,<lgl>,<lgl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
22,ENSG00000100211,38656639,37520000,41680000,Exc_mega_eQTL,image_Aging3,mrash,TRUE,FALSE,0.25300727,9.800927e-49,2.918310,0.0035193435,eQTL,chr22_37320379_39179317
22,ENSG00000100211,38656639,37520000,41680000,Exc_mega_eQTL,image_Aging3,susie,TRUE,TRUE,0.26004704,2.979502e-50,2.976690,0.0029137891,eQTL,chr22_37320379_39179317
22,ENSG00000100211,38656639,37520000,41680000,Inh_mega_eQTL,image_Aging3,enet,TRUE,FALSE,0.05629653,4.239173e-11,2.972824,0.0029507328,eQTL,chr22_37320379_39179317
22,ENSG00000100211,38656639,37520000,41680000,Inh_mega_eQTL,image_Aging3,lasso,TRUE,FALSE,0.05436679,9.118908e-11,2.947601,0.0032025053,eQTL,chr22_37320379_39179317
22,ENSG00000100211,38656639,37520000,41680000,Inh_mega_eQTL,image_Aging3,susie,TRUE,FALSE,0.05917176,1.351464e-11,3.033756,0.0024152949,eQTL,chr22_37320379_39179317
22,ENSG00000100211,38656639,37520000,41680000,Oli_mega_eQTL,image_Aging3,enet,TRUE,FALSE,0.01300504,1.122643e-03,3.456561,0.0005471144,eQTL,chr22_37320379_39179317
22,ENSG00000100211,38656639,37520000,41680000,Oli_mega_eQTL,image_Aging3,lasso,TRUE,FALSE,0.01553025,4.080140e-04,3.265957,0.0010909468,eQTL,chr22_37320379_39179317
22,ENSG00000100211,38656639,37520000,41680000,Oli_mega_eQTL,image_Aging3,mrash,TRUE,TRUE,0.01664078,2.620097e-04,3.161934,0.0015672496,eQTL,chr22_37320379_39179317
22,ENSG00000100211,38656639,37520000,41680000,Oli_mega_eQTL,image_Aging3,susie,TRUE,FALSE,0.01550580,4.120187e-04,3.094809,0.0019693946,eQTL,chr22_37320379_39179317


In [9]:
image_Aging3_twas |> filter(molecular_id == "ENSG00000100211" & str_detect(type, "sQTL") & twas_pval < 0.025)

chr,molecular_id,TSS,start,end,context,gwas_study,method,is_imputable,is_selected_method,rsq_cv,pval_cv,twas_z,twas_pval,type,block
<int>,<chr>,<int>,<int>,<int>,<chr>,<chr>,<chr>,<lgl>,<lgl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
22,ENSG00000100211,38656639,37520000,41680000,ROSMAP_AC_sQTL_chr22:38670989:38673158:clu_143019_+:UP:ENSG00000100211,image_Aging3,enet,TRUE,FALSE,0.045599817,7.144613e-08,3.286375,0.001014857,unproductive_sQTL,chr22_37320379_39179317
22,ENSG00000100211,38656639,37520000,41680000,ROSMAP_AC_sQTL_chr22:38670989:38673158:clu_143019_+:UP:ENSG00000100211,image_Aging3,lasso,TRUE,FALSE,0.038390739,7.406958e-07,3.248037,0.001162042,unproductive_sQTL,chr22_37320379_39179317
22,ENSG00000100211,38656639,37520000,41680000,ROSMAP_AC_sQTL_chr22:38670989:38673158:clu_143019_+:UP:ENSG00000100211,image_Aging3,mrash,TRUE,FALSE,0.034363084,2.731748e-06,2.760038,0.005779468,unproductive_sQTL,chr22_37320379_39179317
22,ENSG00000100211,38656639,37520000,41680000,ROSMAP_AC_sQTL_chr22:38670989:38673158:clu_143019_+:UP:ENSG00000100211,image_Aging3,susie,TRUE,TRUE,0.052347608,7.955851e-09,3.097258,0.001953198,unproductive_sQTL,chr22_37320379_39179317
22,ENSG00000100211,38656639,37520000,41680000,ROSMAP_DLPFC_sQTL_chr22:38670989:38671069:clu_171691_+:PR:ENSG00000100211,image_Aging3,mrash,TRUE,FALSE,0.008680093,4.668280e-03,2.709489,0.006738689,productive_sQTL,chr22_37320379_39179317
22,ENSG00000100211,38656639,37520000,41680000,ROSMAP_DLPFC_sQTL_chr22:38671188:38673158:clu_171691_+:PR:ENSG00000100211,image_Aging3,enet,TRUE,TRUE,0.020153643,3.096515e-05,-2.400003,0.016394959,productive_sQTL,chr22_37320379_39179317
22,ENSG00000100211,38656639,37520000,41680000,ROSMAP_DLPFC_sQTL_chr22:38671188:38673158:clu_171691_+:PR:ENSG00000100211,image_Aging3,lasso,TRUE,FALSE,0.017962080,7.982218e-05,-2.434093,0.014929163,productive_sQTL,chr22_37320379_39179317
22,ENSG00000100211,38656639,37520000,41680000,ROSMAP_DLPFC_sQTL_chr22:38671188:38673158:clu_171691_+:PR:ENSG00000100211,image_Aging3,mrash,TRUE,FALSE,0.015062845,2.808700e-04,-3.052975,0.002265849,productive_sQTL,chr22_37320379_39179317
22,ENSG00000100211,38656639,37520000,41680000,ROSMAP_DLPFC_sQTL_chr22:38671188:38673158:clu_171691_+:PR:ENSG00000100211,image_Aging3,susie,TRUE,FALSE,0.013833902,4.799718e-04,-2.996968,0.002726793,productive_sQTL,chr22_37320379_39179317


In [18]:
# FAM227A
image_Aging3_twas |> filter(molecular_id == "ENSG00000184949"& twas_pval < 4.2*1e-3) 


chr,molecular_id,TSS,start,end,context,gwas_study,method,is_imputable,is_selected_method,rsq_cv,pval_cv,twas_z,twas_pval,type,block
<int>,<chr>,<int>,<int>,<int>,<chr>,<chr>,<chr>,<lgl>,<lgl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
22,ENSG00000184949,38656628,37520000,41680000,Ast_mega_eQTL,image_Aging3,enet,TRUE,FALSE,0.071446837,9.607935e-14,-2.881221,0.0039613717,eQTL,chr22_37320379_39179317
22,ENSG00000184949,38656628,37520000,41680000,Ast_mega_eQTL,image_Aging3,mrash,TRUE,FALSE,0.060193789,8.716063e-12,-2.867472,0.0041376553,eQTL,chr22_37320379_39179317
22,ENSG00000184949,38656628,37520000,41680000,Ast_mega_eQTL,image_Aging3,susie,TRUE,TRUE,0.073476967,4.241631e-14,-3.023135,0.0025017065,eQTL,chr22_37320379_39179317
22,ENSG00000184949,38656628,37520000,41680000,Exc_mega_eQTL,image_Aging3,enet,TRUE,FALSE,0.304052408,4.519029e-60,-3.355575,0.0007920025,eQTL,chr22_37320379_39179317
22,ENSG00000184949,38656628,37520000,41680000,Exc_mega_eQTL,image_Aging3,lasso,TRUE,FALSE,0.301898946,1.411444e-59,-3.389685,0.0006997290,eQTL,chr22_37320379_39179317
22,ENSG00000184949,38656628,37520000,41680000,Exc_mega_eQTL,image_Aging3,mrash,TRUE,FALSE,0.300243464,3.379700e-59,-3.213027,0.0013134375,eQTL,chr22_37320379_39179317
22,ENSG00000184949,38656628,37520000,41680000,Exc_mega_eQTL,image_Aging3,susie,TRUE,TRUE,0.305954501,1.647784e-60,-3.114060,0.0018453186,eQTL,chr22_37320379_39179317
22,ENSG00000184949,38656628,37520000,41680000,Inh_mega_eQTL,image_Aging3,enet,TRUE,FALSE,0.189531652,1.304690e-35,-3.108888,0.0018779259,eQTL,chr22_37320379_39179317
22,ENSG00000184949,38656628,37520000,41680000,Inh_mega_eQTL,image_Aging3,lasso,TRUE,FALSE,0.190959897,6.804336e-36,-3.260991,0.0011102342,eQTL,chr22_37320379_39179317


In [10]:
#tomm22
image_Aging3_twas |> filter(molecular_id == "ENSG00000100216")

chr,molecular_id,TSS,start,end,context,gwas_study,method,is_imputable,is_selected_method,rsq_cv,pval_cv,twas_z,twas_pval,type,block
<int>,<chr>,<int>,<int>,<int>,<chr>,<chr>,<chr>,<lgl>,<lgl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
22,ENSG00000100216,38681956,37520000,41680000,ROSMAP_AC_sQTL_chr22:38682095:38682326:clu_143020_+:UP:ENSG00000100216,image_Aging3,enet,TRUE,TRUE,0.019697963,3.207700e-04,1.283699349,0.1992472,unproductive_sQTL,chr22_37320379_39179317
22,ENSG00000100216,38681956,37520000,41680000,ROSMAP_AC_sQTL_chr22:38682095:38682326:clu_143020_+:UP:ENSG00000100216,image_Aging3,lasso,TRUE,FALSE,0.018227320,5.193575e-04,1.016979250,0.3091633,unproductive_sQTL,chr22_37320379_39179317
22,ENSG00000100216,38681956,37520000,41680000,ROSMAP_AC_sQTL_chr22:38682095:38682326:clu_143020_+:UP:ENSG00000100216,image_Aging3,mrash,TRUE,FALSE,-0.001650429,9.284111e-01,0.861645956,0.3888824,unproductive_sQTL,chr22_37320379_39179317
22,ENSG00000100216,38681956,37520000,41680000,ROSMAP_AC_sQTL_chr22:38682095:38682326:clu_143020_+:UP:ENSG00000100216,image_Aging3,susie,TRUE,FALSE,-0.001629170,8.852787e-01,-1.397438369,0.1622818,unproductive_sQTL,chr22_37320379_39179317
22,ENSG00000100216,38681956,37520000,41680000,ROSMAP_PCC_sQTL_chr22:38682441:38682878:clu_131092_+:PR:ENSG00000100216,image_Aging3,enet,TRUE,FALSE,0.033722842,5.364268e-05,0.632070324,0.5273409,productive_sQTL,chr22_37320379_39179317
22,ENSG00000100216,38681956,37520000,41680000,ROSMAP_PCC_sQTL_chr22:38682441:38682878:clu_131092_+:PR:ENSG00000100216,image_Aging3,lasso,TRUE,TRUE,0.037880091,1.947481e-05,0.742316708,0.4578955,productive_sQTL,chr22_37320379_39179317
22,ENSG00000100216,38681956,37520000,41680000,ROSMAP_PCC_sQTL_chr22:38682441:38682878:clu_131092_+:PR:ENSG00000100216,image_Aging3,mrash,TRUE,FALSE,0.013544095,7.766477e-03,-1.066012664,0.2864179,productive_sQTL,chr22_37320379_39179317
22,ENSG00000100216,38681956,37520000,41680000,ROSMAP_PCC_sQTL_chr22:38682441:38682878:clu_131092_+:PR:ENSG00000100216,image_Aging3,susie,TRUE,FALSE,0.022430653,8.507467e-04,-0.008609546,0.9931307,productive_sQTL,chr22_37320379_39179317
22,ENSG00000100216,38681956,37520000,41680000,ROSMAP_PCC_sQTL_chr22:38682996:38683766:clu_131093_+:PR:ENSG00000100216,image_Aging3,enet,TRUE,FALSE,NA,NA,0.865206869,0.3869253,productive_sQTL,chr22_37320379_39179317


In [23]:
#JOSD1
image_Aging3_twas |> filter(molecular_id == "ENSG00000100221"  & twas_pval < 3.125*1e-3)


chr,molecular_id,TSS,start,end,context,gwas_study,method,is_imputable,is_selected_method,rsq_cv,pval_cv,twas_z,twas_pval,type,block
<int>,<chr>,<int>,<int>,<int>,<chr>,<chr>,<chr>,<lgl>,<lgl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
22,ENSG00000100221,38701555,37520000,41680000,Exc_mega_eQTL,image_Aging3,susie,TRUE,FALSE,0.02776029,3.226394e-06,3.268304,0.001081939,eQTL,chr22_37320379_39179317


In [11]:
# SUN2
image_Aging3_twas |> filter(molecular_id == "ENSG00000100242")



chr,molecular_id,TSS,start,end,context,gwas_study,method,is_imputable,is_selected_method,rsq_cv,pval_cv,twas_z,twas_pval,type,block
<int>,<chr>,<int>,<int>,<int>,<chr>,<chr>,<chr>,<lgl>,<lgl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
22,ENSG00000100242,38794142,37520000,41680000,Exc_mega_eQTL,image_Aging3,enet,TRUE,FALSE,0.009066422,5.557979e-03,-3.461783,0.0005366092,eQTL,chr22_37320379_39179317
22,ENSG00000100242,38794142,37520000,41680000,Exc_mega_eQTL,image_Aging3,lasso,TRUE,TRUE,0.011692147,1.906827e-03,-3.590285,0.0003303166,eQTL,chr22_37320379_39179317
22,ENSG00000100242,38794142,37520000,41680000,Exc_mega_eQTL,image_Aging3,mrash,TRUE,FALSE,-0.001171303,7.094549e-01,-2.953230,0.0031446779,eQTL,chr22_37320379_39179317
22,ENSG00000100242,38794142,37520000,41680000,Exc_mega_eQTL,image_Aging3,susie,TRUE,FALSE,-0.001360127,9.860392e-01,-1.664106,0.0960912286,eQTL,chr22_37320379_39179317
22,ENSG00000100242,38794142,37520000,41680000,Oli_mega_eQTL,image_Aging3,enet,TRUE,FALSE,0.154820268,6.394145e-29,1.711819,0.0869301099,eQTL,chr22_37320379_39179317
22,ENSG00000100242,38794142,37520000,41680000,Oli_mega_eQTL,image_Aging3,lasso,TRUE,FALSE,0.160302114,5.752480e-30,1.745916,0.0808255459,eQTL,chr22_37320379_39179317
22,ENSG00000100242,38794142,37520000,41680000,Oli_mega_eQTL,image_Aging3,mrash,TRUE,FALSE,0.168811518,1.328129e-31,1.644783,0.1000145494,eQTL,chr22_37320379_39179317
22,ENSG00000100242,38794142,37520000,41680000,Oli_mega_eQTL,image_Aging3,susie,TRUE,TRUE,0.180928661,5.816888e-34,1.419835,0.1556558218,eQTL,chr22_37320379_39179317
